In [ ]:
!pip install pyngrok
!pip install streamlit
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 138.5 MB/s eta 0:00:00


In [ ]:
import os
from threading import Thread
from pyngrok import ngrok
import streamlit as st

ngrok.set_auth_token('your auth token') #put your ngrok auth token. If dont have, feel free to create ngrok acc. It is free
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system('streamlit run /content/app.py --server.port 8501')

In [ ]:
%%writefile app.py
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForText2Image
from diffusers.utils import load_image
import torch
import streamlit as st

def initiate():
  pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16") #change model or quantization here if needed)
  pipe.to("cuda")
  st.session_state["image_gen"] = pipe


def gen(prompt):
  pipe= st.session_state["image_gen"]
  image = pipe(prompt, num_inference_steps=10, strength=0.5, guidance_scale=0.0).images[0] # change parameter here if needed)
  image.save("generate.png")
  return image


st.set_page_config(page_title="Image Generator", page_icon="🪄")
st.title("🪄 Lightweight Image Generator 🪄")

# Sidebar for settings
st.sidebar.header("Lighweight image-gen")
model = st.sidebar.text_input("Model Diffusion used", value="Stable Diffusion Turbo")

if st.sidebar.button("Start"):
  initiate()


# Store chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for m in st.session_state.messages:
    with st.chat_message(m["role"]):
        if m["type"] == "image":
            st.image(m["content"],caption="Generated Image")
        else:
          st.markdown(m["content"])

# User input
if prompt := st.chat_input("Type your message…"):
  st.chat_message("user").markdown(prompt)
  response=gen(prompt)

  with st.chat_message("assistant"):
    img=st.image("generate.png", caption="Generated Image")
    url="generate.png"

  st.session_state.messages.append({"role": "user", "type":"text" ,"content": prompt})
  st.session_state.messages.append({"role": "assistant", "type": "image","content": response})




Overwriting app.py


In [ ]:
thread = Thread(target=run_streamlit)
thread.start()
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://f5a5dfdb71bf.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill() # kill unused services as free ngrok have up to 3 simulationous services at a time.